In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, Dropout, Activation, concatenate
from tensorflow.keras.models import Model

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
data = np.load('/content/drive/MyDrive/MedicalImaging/es4-denoising_CNN/dataTrue.npy')
data_highNoise = np.load('/content/drive/MyDrive/MedicalImaging/es4-denoising_CNN/dataNoise_high.npy')
data_smallNoise = np.load('/content/drive/MyDrive/MedicalImaging/es4-denoising_CNN/dataNoise_small.npy')
data_mixedNoise_values = np.load('/content/drive/MyDrive/MedicalImaging/es4-denoising_CNN/dataNoise_mixed_values.npy')
data_mixedNoise = np.load('/content/drive/MyDrive/MedicalImaging/es4-denoising_CNN/dataNoise_mixed.npy')

print(data.shape, data_highNoise.shape, data_smallNoise.shape, data_mixedNoise_values.shape, data_mixedNoise.shape)

(128, 128, 2000) (128, 128, 2000) (128, 128, 2000) (2000,) (128, 128, 2000)


In [3]:
# Reshape data to right format
data = data.transpose(2, 0, 1)
data_highNoise = data_highNoise.transpose(2, 0, 1)
data_smallNoise = data_smallNoise.transpose(2, 0, 1)
data_mixedNoise = data_mixedNoise.transpose(2, 0, 1)

print(data.shape)
print(data_highNoise.shape)
print(data_smallNoise.shape)
print(data_mixedNoise.shape)

(2000, 128, 128)
(2000, 128, 128)
(2000, 128, 128)
(2000, 128, 128)


In [7]:
# Covolutional Block
def conv_block_2d(input, n_filters, dropout_rate):
    x = Conv2D(n_filters, 3, padding='same')(input)
    x = Activation('relu')(x)
    return x

# Encoder Block
def encoder_block_2d(input, num_filters, dropout_rate):
    x = conv_block_2d(input, num_filters, dropout_rate)
    p = MaxPooling2D((2, 2))(x)
    return x, p

# Decoder Block
def decoder_block_2d(input, skip_features, num_filters, dropout_rate):
    x = Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input)
    x = concatenate([x, skip_features])  # Connessione con lo skip connection dell'encoder
    x = conv_block_2d(x, num_filters, dropout_rate)
    return x

# Simple U-Net model for denoising
def build_unet_denoising(input_shape):
    inputs = Input(input_shape)

    # Encoder
    s1, p1 = encoder_block_2d(inputs, 16, 0)
    s2, p2 = encoder_block_2d(p1, 32, 0)
    s3, p3 = encoder_block_2d(p2, 64, 0)

    # Bottleneck
    b1 = conv_block_2d(p3, 128, 0)

    # Decoder
    d1 = decoder_block_2d(b1, s3, 64, 0)
    d2 = decoder_block_2d(d1, s2, 32, 0)
    d3 = decoder_block_2d(d2, s1, 16, 0)

    # Output layer
    outputs = Conv2D(1, 1, padding='same', activation='linear')(d3)

    model = Model(inputs, outputs, name='Denoising_U-Net')
    return model


input_shape = (128, 128, 1)
model1 = build_unet_denoising(input_shape)

model1.compile(optimizer='adam', loss='mse')
model1.summary()

Model: "Denoising_U-Net"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 128, 128, 1)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_8 (Conv2D)         │ (None, 128, 128, 16)   │            160 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_7 (Activation) │ (None, 128, 128, 16)   │              0 │ conv2d_8[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_3           │ (None, 64, 64, 16)     │              0 │ activation_7[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_9 (Conv2D)         │ (None, 64, 64, 32)     │          4,640 │ max_pooling2d_3[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_8 (Activation) │ (None, 64, 64, 32)     │              0 │ conv2d_9[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_4           │ (None, 32, 32, 32)     │              0 │ activation_8[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_10 (Conv2D)        │ (None, 32, 32, 64)     │         18,496 │ max_pooling2d_4[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_9 (Activation) │ (None, 32, 32, 64)     │              0 │ conv2d_10[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_5           │ (None, 16, 16, 64)     │              0 │ activation_9[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_11 (Conv2D)        │ (None, 16, 16, 128)    │         73,856 │ max_pooling2d_5[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_10             │ (None, 16, 16, 128)    │              0 │ conv2d_11[0][0]        │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_transpose_3        │ (None, 32, 32, 64)     │         32,832 │ activation_10[0][0]    │
│ (Conv2DTranspose)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_3             │ (None, 32, 32, 128)    │              0 │ conv2d_transpose_3[0]… │
│ (Concatenate)             │                        │                │ activation_9[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_12 (Conv2D)        │ (None, 32, 32, 64)     │         73,792 │ concatenate_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_11             │ (None, 32, 32, 64)     │              0 │ conv2d_12[0][0]        │
│ (Activation)         

 Total params: 237,169 (926.44 KB)

 Trainable params: 237,169 (926.44 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
X_train, X_val, y_train, y_val = train_test_split(data_mixedNoise, data, test_size=0.2, random_state=77)
history1 = model1.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=32, verbose=1)

Epoch 1/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 8s 55ms/step - loss: 0.1072 - val_loss: 0.0051
Epoch 2/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 0.0043 - val_loss: 0.0023
Epoch 3/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 0.0022 - val_loss: 0.0018
Epoch 4/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - loss: 0.0018 - val_loss: 0.0016
Epoch 5/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - loss: 0.0017 - val_loss: 0.0014
Epoch 6/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 0.0016 - val_loss: 0.0014
Epoch 7/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - loss: 0.0015 - val_loss: 0.0013
Epoch 8/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - loss: 0.0014 - val_loss: 0.0013
Epoch 9/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 0.0014 - val_loss: 0.0012
Epoch 10/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 0.0014 - val_loss: 0.0012
Epoch 11/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - loss: 0.0013 - val_loss: 0.0011
Epoch 12/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - loss: 0.0

In [11]:
predSmall = model1.predict(data_smallNoise).squeeze()
predHigh = model1.predict(data_highNoise).squeeze()

mseSmall = np.mean((predSmall - data_smallNoise) ** 2)
mseHigh = np.mean((predHigh - data_highNoise) ** 2)

print("MSE for small noise data:", mseSmall)
print("MSE for high noise data:", mseHigh)

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
MSE for small noise data: 0.0017805513211821946
MSE for high noise data: 0.008945201081932494


In [12]:
X_train, X_val, y_train, y_val = train_test_split(data_smallNoise, data, test_size=0.2, random_state=77)

model2 = build_unet_denoising(input_shape)
model2.compile(optimizer='adam', loss='mse')
history2 = model2.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=32, verbose=1)

Epoch 1/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 8s 54ms/step - loss: 0.0945 - val_loss: 0.0029
Epoch 2/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 7s 36ms/step - loss: 0.0023 - val_loss: 0.0010
Epoch 3/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 8.3421e-04 - val_loss: 4.9695e-04
Epoch 4/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - loss: 4.7139e-04 - val_loss: 4.0009e-04
Epoch 5/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - loss: 3.8246e-04 - val_loss: 3.5610e-04
Epoch 6/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - loss: 3.5847e-04 - val_loss: 3.3992e-04
Epoch 7/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 3.5634e-04 - val_loss: 3.2799e-04
Epoch 8/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 3.3674e-04 - val_loss: 3.1337e-04
Epoch 9/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - loss: 3.1439e-04 - val_loss: 3.0503e-04
Epoch 10/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 3.0785e-04 - val_loss: 2.9938e-04
Epoch 11/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - loss: 3.1489e-04 - val_loss: 3.

In [17]:
predMixed = model2.predict(data_mixedNoise).squeeze()
predHigh = model2.predict(data_highNoise).squeeze()

mseMedium = np.mean((predMixed - data_mixedNoise) ** 2)
mseHigh = np.mean((predHigh - data_highNoise) ** 2)

print("MSE for mixed noise data:", mseSmall)
print("MSE for high noise data:", mseHigh)

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
MSE for mixed noise data: 0.004282408004176306
MSE for high noise data: 0.006983599341356796


In [16]:
X_train, X_val, y_train, y_val = train_test_split(data_highNoise, data, test_size=0.2, random_state=77)

model3 = build_unet_denoising(input_shape)
model3.compile(optimizer='adam', loss='mse')
history3 = model3.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=32, verbose=1)

Epoch 1/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 8s 52ms/step - loss: 0.0504 - val_loss: 0.0034
Epoch 2/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 0.0030 - val_loss: 0.0019
Epoch 3/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - loss: 0.0019 - val_loss: 0.0015
Epoch 4/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - loss: 0.0015 - val_loss: 0.0013
Epoch 5/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 0.0013 - val_loss: 0.0012
Epoch 6/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 0.0012 - val_loss: 0.0012
Epoch 7/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 0.0012 - val_loss: 0.0011
Epoch 8/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - loss: 0.0011 - val_loss: 0.0010
Epoch 9/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - loss: 0.0011 - val_loss: 0.0010
Epoch 10/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - loss: 0.0010 - val_loss: 9.9174e-04
Epoch 11/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - loss: 0.0010 - val_loss: 9.7311e-04
Epoch 12/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - l

In [18]:
predMixed = model3.predict(data_mixedNoise).squeeze()
predSmall = model3.predict(data_smallNoise).squeeze()

mseMedium = np.mean((predMixed - data_mixedNoise) ** 2)
mseSMall = np.mean((predSmall - data_smallNoise) ** 2)

print("MSE for mixed noise data:", mseSmall)
print("MSE for small noise data:", mseSMall)

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
MSE for mixed noise data: 0.004282408004176306
MSE for small noise data: 0.0018280407585425094
